## Database Design

- Data types
- Online Analytical Processing vs Online Transaction Processing
- ACID transactions ?
- Traditional DBs vs Datawarehouses vs Data Lakes
- ETL vs ELT
- Database models and schemas
- Relational model
- Dimensional modeling
- Normalisation and normal forms
- Database views
- Database roles
- Database management systems
- Partitioning / sharding

OLAP vs OLTP
 - Online analytical processing vs online transaction processing
 - Figure out business requirements first

Structuring data
 - Structured data e.g. follows a schema, has defined data types and relationships (SQL, tables in relational DBs)
 - Unstructured data are schema less and the make up most of the data in the worlds e.g. photos, chat logs, mp3
 - Semi-structured data has self-describing structure but does not follow a larger schema e.g. NoSQL, XML, JSON

Traditional databases e.g. operational databases that use OLTP, are needed for storing relational structured data (relational schema).
 - They used to be enough for data storage
 - Then as data analytics took off, data warehouses were popularised for analysing structured data (OLAP)
 - In the age of big data, data lakes needed for storing data of all structures (flexible and scalable)

DWH are optimised for analytics - OLAP
 - They contain data from multiple resources
 - Are usually read-only
 - The use MPP (massively parallel processing)
 - Typically have a denormalised schema and dimensional modelling
 - Schema-on-write (predefined schema, same as traditional databases)
 - Uses ETL (extract-transform-load)
 - AWS Redshift, Google BigQuery, Azure SQL DWH

Data lakes
 - Storage is cheaper - uses object storage instead of file storage
 - They can reach petabytes (1k terabyte)
 - Schema-on-read because the schema is created as data is read
 - Must be organised and cataloged well, otherwise they become data swamps
 - Uses ELT

Database design: how data is logically organised and store, how is data going to be read and updated

Database models: high-level specifications for database structure
 - The relational model is the most popular model
 - Other models: NoSQL models, object-oriented model, network model
 - Schema is the databases blueprint, it defines tables, fields, relationships, indexes and views.
 - Insertion of data follows the schema

Dimensional modelling is the adaptation of the relational model for DWH design
 - Optimised for OLAP queries to aggregate data (i.e. not update it)
 - Built using the star schema/easy to interpret and extend
 - Includes fact tables that hold records of a metric and change regularly
 - Includes dimensions tables that hold description of attributes and do not change as often
 
 
![title](assets/db_design/oltp_vs_olap.png)

Dimensional modelling
 - One of its manifestations is the Star Schema.

Fact and dimension tables.
 - Fact tables hold records of metrics.
 - Dimension tables describe these records.

![image.png](assets/db_design/star_schema.png)
Snowflake schema is an extension of the star schema.
 - The snowflake schema extends in more than one dimensions vs the star schema which extends in a single dimensions.
 - In the snowflake schema the dimension tables are normalised.

![image.png](assets/db_design/snowflake_schema.png)

Normalisation is a technique that divides tables into smaller tables and connects them via relationships.
 - The goal of normalisation is to reduce data redundancy and increase data integrity by identifying repeating groups of data and creating new tables for them.
 - Several forms of normalisation exists (1NF, 2NF, 3NF ...).
 - Normalised DBs will look more complicated and you’d need lengthier more complex queries to retrieve data (more CPU requirements).
 - So why to use them? They save space (have more tables, but far less records) as there is not repeated information).

 - Therefore you have reduced data redundancy.

 - More: data integrity through enforced data consistency (e.g. a record for ‘California’ won’t be found as ‘CA’ or ‘california’) this is called referential integrity
 - More: as records duplicates are reduced, it is safer to update, remove and insert data
 - More: as tables are smaller and organised by object, it’s easier to alter the database schema if needed (e.g. extend a table)

![image.png](assets/db_design/normalisation.png)

OLTP (e.g. operational databases) is write-intensive: prioritise quicker and safer insertion of data (typically highly normalised)
OLAP (e.g. data warehouses) is read-intensive: prioritise quicker queries for analytics (typically less normalised)


More formal Normalisation definition:

 - 1NF: records are unique (no duplicate rows) and each cell must hold one value

 - 2NF: must satisfy 1NF and
  - if primary key is one column then it automatically satisfied 2NF
  - else if primary key is a composite primary key (2 or more columns establish the key) then each non-key column must be dependent on all the keys

 - 3NF: must satisfy 2NF and have no transitive dependencies (i.e. non-key columns can’t depend on other non-key columns)
![image.png](assets/db_design/normal_forms.png)

Risk of non-normalisation:
 - Update - insertion - deletion data anomalies (i.e. too many records to change)



In [ ]:
# -- Add the book_id foreign key
ALTER TABLE fact_booksales ADD CONSTRAINT sales_book
    FOREIGN KEY (book_id) REFERENCES dim_book_star (book_id);
    
# -- Add the time_id foreign key
ALTER TABLE fact_booksales ADD CONSTRAINT sales_time
    FOREIGN KEY (time_id) REFERENCES dim_time_star (time_id);
    
# -- Add the store_id foreign key
ALTER TABLE fact_booksales ADD CONSTRAINT sales_store
    FOREIGN KEY (store_id) REFERENCES dim_store_star (store_id);


In [ ]:
# -- Create a new table for dim_author with an author column
CREATE TABLE dim_author (
    author varchar(256)  NOT NULL
);

# -- Insert authors 
INSERT INTO dim_author
SELECT DISTINCT author FROM dim_book_star;

# -- Add a primary key 
ALTER TABLE dim_author ADD COLUMN author_id SERIAL PRIMARY KEY;

# -- Output the new table
SELECT * FROM dim_author;

In [ ]:
# -- Output each state and their total sales_amount
SELECT dim_store_star.state, SUM(sales_amount)
FROM fact_booksales
# -- Join to get book information
    JOIN dim_book_star on dim_book_star.book_id = fact_booksales.book_id
# -- Join to get store information
    JOIN dim_store_star on dim_store_star.store_id = fact_booksales.store_id
# -- Get all books with in the novel genre
WHERE  
    dim_book_star.genre = 'novel'
# -- Group results by state
GROUP BY
    dim_store_star.state;

In [ ]:
# -- Output each state and their total sales_amount
SELECT dim_state_sf.state, SUM(sales_amount)
FROM fact_booksales
#     -- Joins for genre
    JOIN dim_book_sf on fact_booksales.book_id = dim_book_sf.book_id
    JOIN dim_genre_sf on dim_book_sf.genre_id = dim_genre_sf.genre_id
#     -- Joins for state 
    JOIN dim_store_sf on fact_booksales.store_id = dim_store_sf.store_id 
    JOIN dim_city_sf on dim_store_sf.city_id = dim_city_sf.city_id
    JOIN dim_state_sf on  dim_city_sf.state_id = dim_state_sf.state_id
# -- Get all books with in the novel genre and group the results by state
WHERE  
    dim_genre_sf.genre = 'novel'
GROUP BY
    dim_state_sf.state;

In [ ]:
# -- Add a continent_id column with default value of 1
ALTER TABLE dim_country_sf
ADD continent_id int NOT NULL DEFAULT(1);

# -- Add the foreign key constraint
ALTER TABLE dim_country_sf ADD CONSTRAINT country_continent
   FOREIGN KEY (continent_id) REFERENCES dim_continent_sf(continent_id);

# -- Output updated table
SELECT * FROM dim_country_sf;

Views are virtual tables not part of the physical schema
 - The query and not the data are stored in memory
 - The can be queried like normal tables

The benefits of views are
 - control access to information (hide sensitive columns, restrict what users can see)
 - don’t take up storage
 - masks complexity of queries (useful for highly normalised schemas)
 
![image.png](assets/db_design/tables_vs_views.png)

In [ ]:
#To create a view:
CREATE VIEW  name AS query

#To look at all available views:
SELECT * FROM INFORMATION_SCHEMA.views

#Excluding system views
#-- Get all non-systems views
SELECT * FROM INFORMATION_SCHEMA.views
WHERE table_schema NOT IN ('pg_catalog', 'information_schema');

Managing views
 - GRANT / REVOKE privileges ON object TO/FROM role
 - Privileges: SELECT, INSERT, UPDATE, DELETE
 - Objects: table, view, schema
 - Role: db user or db users group

To be updatable a view must have no special clauses and be made of a single table.

Materialised views
 - store a query’s results as a table on disk (instead of virtualised tables)
 - updated less frequently
 - good for long queries when results are needed relatively quickly
 
![image.png](assets/db_design/mat_vs_non_mat_views.png)

Roles - entities to manage access permissions

Contains information about login, database creation, tables write-in

Can be assigned to one or more users

Are global across database cluster installation

In [ ]:
# Empty role
CREATE ROLE data_analyst;
# Roles with attributes
CREATE ROLE intern WITH PASSWORD ‘psw’  VALID UNTIL ‘2020-01-01’
CREATE ROLE admin CREATEDB;

# To change attribute for a role already created
ALTER ROLE admin CREATEROLE;

GRANT/REVOKE  UPDATE ON table TO/FROM role_name;
SELECT, INSERT, UPDATE, DELETE, TRUNCATE, REFERENCES, TRIGGER, CREATE, CONNECT, TEMPORARY, EXECUTE, USAGE


# There are also group roles which encompass a group of user roles.

CREATE ROLE data_scientist
# -- Create a role for Marta
CREATE ROLE marta WITH LOGIN;
# -- Create an admin role
CREATE ROLE  admin WITH CREATEDB CREATEROLE;

# -- Grant data_scientist update and insert privileges
GRANT UPDATE, INSERT ON long_reviews TO data_scientist;

# -- Give Marta's role a password
ALTER ROLE marta WITH PASSWORD 's3cur3p@ssw0rd';

# -- Add Marta to the data scientist group
GRANT data_scientist TO marta;

# -- Remove Marta from the data scientist group
REVOKE data_scientist FROM marta;

Partitioning

 - When tables grow queries become very slow because indices don’t fit into memory
 - Tables are split into separate parts
 - Physically data are distributed over several entities

Vertical partitioning - table is split vertically by columns

Horizontal partitioning - table is split by rows (e.g. timestamp)

Sharding is the horizontal partitioning of a table over several machines

In [ ]:
# -- Create a new table called film_descriptions
CREATE TABLE film_descriptions (
    film_id INT,
    long_description TEXT
);

# -- Copy the descriptions from the film table
INSERT INTO film_descriptions
SELECT film_id, long_description FROM film;
    
# -- Drop the descriptions from the original table
ALTER TABLE film DROP COLUMN long_description;

# -- Join to view the original table
SELECT * FROM film 
JOIN film_descriptions USING(film_id);

# -- Create a new table called film_partitioned
CREATE TABLE film_partitioned (
  film_id INT,
  title TEXT NOT NULL,
  release_year TEXT
)

PARTITION BY LIST (release_year);

# -- Create the partitions for 2019, 2018, and 2017
CREATE TABLE film_2019
  PARTITION OF film_partitioned FOR VALUES IN ('2019');

CREATE TABLE film_2018
  PARTITION OF film_partitioned FOR VALUES IN ('2018');

CREATE TABLE film_2017
  PARTITION OF film_partitioned FOR VALUES IN ('2017');

# -- Insert the data into film_partitioned
INSERT INTO film_partitioned
SELECT film_id, title, release_year FROM film;

# -- View film_partitioned
SELECT * FROM film_partitioned;


DBMS - a system software for creating and maintaining databases

It manages 3 important aspects: the data, the database schema and the database engine

It is an interface between the database and end users or application programs

![image.png](assets/db_design/elt_vs_etl.png)